In [13]:
import requests
from fuzzywuzzy import process

TMDB_API_KEY = 'a76d2f9cc87b26f475a83b8f8253356d'

def get_movie_details_tmdb(movie_name):
    """
    Get movie details (summary, cast) using TMDb API.
    """
    search_url = f"https://api.themoviedb.org/3/search/movie?query={movie_name}&api_key={TMDB_API_KEY}"
    search_response = requests.get(search_url)

    if search_response.status_code == 200:
        search_data = search_response.json()
        if search_data['results']:
            best_match = search_data['results'][0]
            movie_id = best_match['id']
            title = best_match.get('title', 'Unknown Title')
            overview = best_match.get('overview', 'No summary available.')

            cast_url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={TMDB_API_KEY}"
            cast_response = requests.get(cast_url)

            if cast_response.status_code == 200:
                cast_data = cast_response.json()
                cast = [member['name'] for member in cast_data.get('cast', [])[:5]]  # Top 5 cast members
                cast_list = ', '.join(cast) if cast else 'No cast information available.'
            else:
                cast_list = 'Error fetching cast information.'

            return f"Title: {title}\nSummary: {overview}\nCast: {cast_list}"
        else:
            return "Movie not found in TMDb API."
    else:
        return "Error connecting to TMDb API."

def get_movie_summary(movie_name):
    """
    Use fuzzy matching to find the best movie match and then fetch details from TMDb API.
    """
    search_url = f"https://api.themoviedb.org/3/search/movie?query={movie_name}&api_key={TMDB_API_KEY}"
    search_response = requests.get(search_url)

    if search_response.status_code == 200:
        search_data = search_response.json()
        if search_data['results']:
            movie_names = [movie['title'].strip().lower() for movie in search_data['results']]
            best_match, score = process.extractOne(movie_name.lower().strip(), movie_names)
            if score >= 70:
                best_match_movie = next(movie for movie in search_data['results'] if movie['title'].strip().lower() == best_match)
                return get_movie_details_tmdb(best_match_movie['title'])
            else:
                return f"Movie not found with high confidence. Did you mean one of these?\n{', '.join(movie_names[:5])}"
        else:
            return "Movie not found in TMDb API."
    else:
        return "Error connecting to TMDb API."

if __name__ == "__main__":
    movie_name = input("Enter the movie name to get its details: ")
    details = get_movie_summary(movie_name)
    print(f"\nDetails of '{movie_name}':\n{details}")


Enter the movie name to get its details: pirates caribbean

Details of 'pirates caribbean':
Title: True Caribbean Pirates
Summary: Blackbeard, Henry Morgan, Ann Bonny and Black Bart Roberts. Larger than life, more dangerous than legend - pirates and buccaneers set sail for plunder. Shot in high definition - True Caribbean Pirates recreates the rise of piracy in the Caribbean and its climactic, inevitable downfall.
Cast: KImberly Adair, Peter Bisson, Andrew Block, James Bolt


In [2]:
!pip install fuzzywuzzy